In [1]:
import numpy as np
from scipy.spatial import distance
import itertools

In [2]:
def reliability(v1, v2):
    """v1/v2: spike signals of two simulations/set of neurons.  Arrays of size # neurons x # time_bins"""
    product_of_norms = (np.linalg.norm(v1, axis=1) * np.linalg.norm(v2, axis=1))
    #mask_to_normalize_non_zero = (product_of_norms != 0)
    reliability = np.multiply(v1 , v2).sum(axis=1)
    reliability=np.divide(reliability,product_of_norms,where=(product_of_norms != 0))
    #reliability[mask_to_normalize_non_zero] = reliability[mask_to_normalize_non_zero] / product_of_norms[
        #mask_to_normalize_non_zero]
    return reliability

def reliability_cell(v1, v2):
    """Computes reliability of two spike trains."""
    product_of_norms=np.linalg.norm(v1) * np.linalg.norm(v2)
    if  product_of_norms> 0.0:
        return np.dot(v1, v2) / product_of_norms
    else:
        return 0.0

# Testing inner function 

In [3]:
#Generate fake data set
k=10 #number of sims 
gids=260#00 #number of cells
tbins=160#00 #number of timebins

ss1=np.random.normal(loc=0.25,scale=0.3,size=(gids,tbins))
ss2=np.random.normal(loc=0.25,scale=0.3,size=(gids,tbins))
ss1=ss1-ss1.mean();ss2=ss2-ss2.mean();

In [4]:
%%time 
rels_noloop=reliability(ss1,ss2)
rels_noloop.shape

CPU times: user 289 µs, sys: 1.23 ms, total: 1.52 ms
Wall time: 953 µs


(260,)

In [5]:
%%time
rels_loop=[reliability_cell(ss1[x,:], ss2[x,:]) for x in range(gids)]
rels_loop=np.array(rels_loop)
rels_loop.shape

CPU times: user 7.43 ms, sys: 6.26 ms, total: 13.7 ms
Wall time: 7.8 ms


(260,)

In [6]:
%%time
x=1-np.array([distance.pdist(np.stack([ss1[x,:],ss2[x,:]]),'cosine')[0] for x in range(gids)])

CPU times: user 7.93 ms, sys: 3.7 ms, total: 11.6 ms
Wall time: 8.27 ms


In [7]:
print("All the same?", np.allclose(rels_loop,rels_noloop)&np.allclose(x,rels_noloop))

All the same? True


In [8]:
#Generate fake data set
k=10 #number of sims 
gids=2600#0 #number of cells
tbins=1600#0 #number of timebins

ss1=np.random.normal(loc=0.25,scale=0.3,size=(gids,tbins))
ss2=np.random.normal(loc=0.25,scale=0.3,size=(gids,tbins))

In [9]:
%%time 
rels_noloop=reliability(ss1,ss2)

CPU times: user 23.6 ms, sys: 20.4 ms, total: 44 ms
Wall time: 41.8 ms


In [10]:
%%time
rels_loop=[reliability_cell(ss1[x,:], ss2[x,:]) for x in range(gids)]
rels_loop=np.array(rels_loop)

CPU times: user 70.6 ms, sys: 16.2 ms, total: 86.8 ms
Wall time: 67.4 ms


In [11]:
%%time
x=1-np.array([distance.pdist(np.stack([ss1[x,:],ss2[x,:]]),'cosine')[0] for x in range(gids)])

CPU times: user 55.6 ms, sys: 294 µs, total: 55.9 ms
Wall time: 55.1 ms


In [12]:
print("All the same?", np.allclose(rels_loop,rels_noloop)&np.allclose(x,rels_noloop))

All the same? True


In [13]:
#Generate fake data set
k=10 #number of sims 
gids=26000 #number of cells
tbins=16000 #number of timebins

ss1=np.random.normal(loc=0.25,scale=0.3,size=(gids,tbins))
ss2=np.random.normal(loc=0.25,scale=0.3,size=(gids,tbins))

In [14]:
%%time 
rels_noloop=reliability(ss1,ss2)

CPU times: user 2.78 s, sys: 2.01 s, total: 4.79 s
Wall time: 4.79 s


In [15]:
%%time
rels_loop=[reliability_cell(ss1[x,:], ss2[x,:]) for x in range(gids)]
rels_loop=np.array(rels_loop)

CPU times: user 5.07 s, sys: 4.53 s, total: 9.6 s
Wall time: 2.54 s


In [16]:
%%time
x=1-np.array([distance.pdist(np.stack([ss1[x,:],ss2[x,:]]),'cosine')[0] for x in range(gids)])

CPU times: user 2.65 s, sys: 197 ms, total: 2.85 s
Wall time: 2.55 s


In [17]:
print("All the same?", np.allclose(rels_loop,rels_noloop)&np.allclose(x,rels_noloop))

All the same? True


#  Testing outer function 

In [18]:
def avg_reliability(v_filt):
    """Computes average reliability between all pairs of trials of a give set.
    v_filt: Array spike trains many simuations of shape N_trials x #neuronss # time_bins"""
    N_trials = v_filt.shape[0]
    avg_rel = np.zeros(v_filt.shape[1])
    for i in range(N_trials):
        x=v_filt[i, :, :]
        for j in np.arange(i+1,N_trials):
            y=v_filt[j, :, :]
            avg_rel = avg_rel + reliability(x, y)
    avg_rel = 2 * avg_rel / (N_trials * (N_trials - 1))
    return avg_rel
def avg_reliability_noloop(v_filt):
    """Computes average reliability between all pairs of spike trains."""
    N_trials = v_filt.shape[0] #number of trials
    avg_rel = 0.0
    for i,j in itertools.combinations(range(N_trials),2):
        avg_rel = avg_rel + reliability_cell(v_filt[i, :], v_filt[j, :])
    avg_rel = 2 * avg_rel / (N_trials * (N_trials - 1))
    return avg_rel
def avg_reliability_scipy(v_filt):
    """Computes average reliability between all pairs of trials of a give set.
        v_filt: Array spike trains many simuations of shape N_trials x #neuronss # time_bins"""
    no_cells=v_filt.shape[1]
    mean_rels=[(1-distance.pdist(v_filt[:,x,:],'cosine')).mean() for x in range(no_cells)]
    return np.array(mean_rels)

In [19]:
#Generate fake data set
k=10 #number of sims 
gids=260#00 #number of cells
tbins=160#00 #number of timebins

ss=np.random.normal(loc=0.25,scale=0.3,size=(k, gids,tbins))

In [20]:
%%time 
rels_noloop=avg_reliability(ss)

CPU times: user 10.3 ms, sys: 0 ns, total: 10.3 ms
Wall time: 9.14 ms


In [21]:
%%time 
rels_loop=np.array([avg_reliability_noloop(ss[:,x,:]) for x in range(gids)])

CPU times: user 182 ms, sys: 31.4 ms, total: 214 ms
Wall time: 180 ms


In [22]:
%%time 
rels_scipy=avg_reliability_scipy(ss)

CPU times: user 8.68 ms, sys: 125 µs, total: 8.81 ms
Wall time: 7.88 ms


In [23]:
print("All the same?", np.allclose(rels_scipy, rels_loop)&np.allclose(rels_noloop, rels_loop))

All the same? True


In [24]:
#Generate fake data set
k=10 #number of sims 
gids=2600 #number of cells
tbins=1600 #number of timebins

ss=np.random.normal(loc=0.25,scale=0.3,size=(k, gids,tbins))

In [25]:
%%time 
rels_noloop=avg_reliability(ss)

CPU times: user 1.39 s, sys: 865 µs, total: 1.39 s
Wall time: 1.39 s


In [26]:
%%time 
rels_loop=np.array([avg_reliability_noloop(ss[:,x,:]) for x in range(gids)])

CPU times: user 2.08 s, sys: 32.3 ms, total: 2.12 s
Wall time: 2.08 s


In [27]:
%%time 
rels_scipy=avg_reliability_scipy(ss)

CPU times: user 270 ms, sys: 769 µs, total: 271 ms
Wall time: 270 ms


In [28]:
print("All the same?", np.allclose(rels_scipy, rels_loop)&np.allclose(rels_noloop, rels_loop))

All the same? True


In [29]:
#Generate fake data set
k=10 #number of sims 
gids=26000//5 #number of cells
tbins=16000//5 #number of timebins

ss=np.random.normal(loc=0.25,scale=0.3,size=(k, gids,tbins))

In [30]:
%%time 
rels_noloop=avg_reliability(ss)

CPU times: user 4.93 s, sys: 3.7 s, total: 8.63 s
Wall time: 8.63 s


In [31]:
%%time 
rels_loop=np.array([avg_reliability_noloop(ss[:,x,:]) for x in range(gids)])

CPU times: user 4.57 s, sys: 4.34 ms, total: 4.58 s
Wall time: 4.58 s


In [32]:
%%time 
rels_scipy=avg_reliability_scipy(ss)

CPU times: user 1.08 s, sys: 51 µs, total: 1.08 s
Wall time: 1.08 s


In [33]:
print("All the same?", np.allclose(rels_scipy, rels_loop)&np.allclose(rels_noloop, rels_loop))

All the same? True


In [34]:
#Generate fake data set
k=10 #number of sims 
gids=26000 #number of cells
tbins=16000 #number of timebins

ss=np.random.normal(loc=0.25,scale=0.3,size=(k, gids,tbins))

In [35]:
%%time 
rels_noloop=avg_reliability(ss)

CPU times: user 2min 5s, sys: 1min 31s, total: 3min 36s
Wall time: 3min 36s


In [36]:
%%time 
rels_loop=np.array([avg_reliability_noloop(ss[:,x,:]) for x in range(gids)])

CPU times: user 3min 1s, sys: 2min 52s, total: 5min 53s
Wall time: 1min 28s


In [37]:
%%time 
rels_scipy=avg_reliability_scipy(ss)

CPU times: user 25.4 s, sys: 208 ms, total: 25.6 s
Wall time: 25.3 s


In [38]:
print("All the same?", np.allclose(rels_scipy, rels_loop)&np.allclose(rels_noloop, rels_loop))

All the same? True
